In [ ]:
from sys import version, version_info
assert version_info >= (3, 7), "Need python >= 3.7, you have:\n{}".format(version_info)

# %load_ext blackcellmagic
# Usage: %%black

from dataclasses import dataclass, field
from typing import List
from queue import Queue, Empty
from random import shuffle, choice

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio
from time import sleep

from IPython.core.display import HTML

# Enable widgets:
# !jupyter nbextension install --py widgetsnbextension --user
# !jupyter nbextension enable widgetsnbextension --user --py

In [ ]:
# @dataclass
# class Pairs:
#     japanese: str
#     english: str
        
@dataclass
class TranslatedSentence:
    english: str
    japanese: str
    furigana_html: str = None
    furigana_anki: str = None
    

@dataclass
class Player:
    name: str
    vocab: List[TranslatedSentence] = field(repr=False)
#     known: List[Pairs] = field(default_factory=lambda :[], init=False)
        
    def __post_init__(self):
        shuffle(self.vocab)
        
        self.to_check = Queue()
        for i in self.vocab:
            self.to_check.put(i)
            
        self.score = {
            "known": 0,
            "failed": 0,
            "skipped": 0,
        }
        
    def update(self, category):
        self.score[category] += 1
       
    def pick_word(self):
        choice = self.to_check.get_nowait()
        return choice
    
    def show_score(self):
        for k in sorted(self.score.keys()):
            print("{:<10}: {:>3}/{:>3} [{:>4.1%}]".format(
                k, self.score[k], len(self.vocab), self.score[k] / len(self.vocab)))

_raw = []

with open("./new_sentences.txt", "r") as f:
    _buff = []
    for i in f.readlines():
        if len(i) < 3:
            # newline
            _raw.append(_buff)
            _buff = []
            continue
            
        _buff.append(i[:-1])

pairs_new = [
    TranslatedSentence(english=eng, japanese=jap)
    for (eng, jap) in _raw
]

In [ ]:
import pykakasi
import re
from typing import List, Tuple

class Furiganator():
    def __init__(self):
        conv_k = pykakasi.kakasi()
        # Japanese to [H(Hiragana)]furigana
        conv_k.setMode("J","HF") 
        # add space
        conv_k.setMode("s", True)  
        self._converter = conv_k.getConverter()

    def _format_html(self, word: str):
        return "<ruby>" + word.replace("[", "<rt>").replace("]", "</rt>") + "</ruby>"
    
    def _format_anki(self, word: str):
        if "[" in word and "]" in word:
            return f" {word} "
        return word

    def __call__(self, text: str) -> Tuple[str, str]:
        """Return the sentence with furigana added, in two formats."""
        text = re.sub("\s+", "", text)
        res = self._converter.do(text)
        words = re.split("\s+", res)
        
        anki_str = "".join(self._format_anki(x) for x in words)
        anki_str = re.sub("^\s+", "", anki_str)
        
        return (
            "".join(self._format_html(x) for x in words),
            anki_str,
        )
    
F = Furiganator()
  
# Add all furigana:
for p in pairs_new:
    _h, _a = F(p.japanese)
    p.furigana_html = _h
    p.furigana_anki = _a

In [ ]:
HTML(
    pairs_new[0].furigana_html
)

In [ ]:
s = []

for p in pairs_new:
    s.extend([p.english, p.furigana_anki, ""])
    
print("\n".join(s))

In [ ]:
%gui asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # TODO: learn how to use futures proprely
        try:
            future.result()
        except asyncio.InvalidStateError:
            future.set_result(0)
    widget.on_click(getvalue)
    return future



In [ ]:
NEW = True
# NEW = False
# _players = ["Dan", "Romeo", "Paul"]
_players = ["Dan", "Paul"]

if NEW:
    pairs = pairs_new
#     vocab_size = len(pairs) * len(_players)
    vocab_size = 87
else:
    pairs = pairs_previous
    vocab_size = 20 * len(_players)


players = [Player(name=name, vocab=pairs) for name in _players]
shuffle(players)

flags = {"japanese": "🇯🇵", "english": "🇬🇧", "aussie": "🇦🇺"}

question = widgets.HTML(
    value="Questions here!",
)

list_players = widgets.Select(
    options=[p.name for p in players],
    value=players[0].name,
    disabled=True
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=vocab_size,
    step=1,
    description="[  0 /{:>3}]".format(vocab_size),
    bar_style='',
    orientation='horizontal'
)

status = widgets.ToggleButtons(
    options=['Known ', 'Unkown ', 'Skipped '],
    disabled=False,
    button_style='',
    icons=["check-circle", "question-circle", "forward"],
)
_mapping = {"Known ": "known", "Unkown ": "failed", "Skipped ": "skipped"}

next_b = widgets.Button(
    description='Next!',
    button_style='',
    icon='check'
)

reveal_b = widgets.Button(
    description='Show answer',
    button_style='',
    icon='question-circle'
)

async def f():
    for i in range(vocab_size):
        player_idx = i % len(players)
        curr_player = players[player_idx]
        list_players.value = curr_player.name
        word = curr_player.pick_word()

        q, a = choice([
            ["japanese", "english"],
            ["english", "japanese"],
        ])
        f_q, f_a = q, a
#         if curr_player.name == "Dan":
#             # Need the right flag
#             f_q = "aussie" if q == "english" else f_q
#             f_a = "aussie" if a == "english" else f_a

#         question.value = 'From <b>{}</b> {} to <b>{}</b> {}, translate:<p><b style="font-size: 24px">{}</b>'.format(
#             q, flags[f_q], a, flags[f_a],
#             getattr(word, q),
#         )
        question.value = '<b style="font-size: 36px">{}</b>'.format(
            word.japanese,
        )
        # Get the furigana
        
        # Reveal the answer
        _ = await wait_for_change(reveal_b, 'value')
#         furigana_answer = kanji_to_html_furigana(word.kanji, conv)
#         question.value = (
#             '<b style="font-size: 36px">{}</b><p><b style="font-size: 36px">{}</b>'
#             '<p><p><p><b style="font-size: 24px">{}</b>'
#             '<p><p><p><b style="font-size: 24px">{}</b>'.format(
#             word.hiragana, word.kanji, furigana_answer, word.english
#         ))
        question.value = (
            '<p><p><p><b style="font-size: 36px">{}</b>'
            '<p><p><p><b style="font-size: 26px">{}</b>'.format(
            word.furigana_html, word.english
        ))
        
        _ = await wait_for_change(next_b, 'value')
        reveal_b.disabled = True
        next_b.disabled = True
        
        curr_player.update(_mapping[status.value])
        
        progress.value += 1
        progress.description = "[{:>3} /{:>3}]".format(progress.value, vocab_size)
        
        # Avoid quick double press
        sleep(0.5)
        status.value = "Known "
        reveal_b.disabled = False
        next_b.disabled = False
        
    print("All done")
    
    for p in players:
        print("~"*80)
        print(f"Name: {p.name}")
        p.show_score()

asyncio.ensure_future(f())

display(progress)
display(question)
display(list_players)
display(status)
display(reveal_b)
next_b


## TODO for sentences notebook

* [x] Better parsing
* [x] Furigana mode?
* [ ] Add a "next" directly option
* [ ] Check hoshigaru/hoshigaaru
* [ ] How ppl pronounce 5fun
* [ ] Natural way to read: 彼のほかに私たち皆が行った。

check yachin yachan
